In [72]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import pretty_midi

In [73]:
# Paths
CLASSIFIER_WEIGHTS = "../models/mood_classifier.h5"
VOCAB_PATH = "artifacts/note_to_idx.pkl"
GENERATED_MIDI = "../output/generated_music/happy_generated.midi"

# Model parameters 
SEQ_LEN = 50
MOODS = ["happy", "sad", "angry", "quiet"]

In [74]:
with open(VOCAB_PATH, "rb") as f:
    note_to_idx = pickle.load(f)

idx_to_note = {i: n for n, i in note_to_idx.items()}
vocab_size = len(note_to_idx)

print("Vocabulary size:", vocab_size)

Vocabulary size: 128


In [75]:
def build_mood_classifier(vocab_size, seq_len, num_classes):
    note_input = Input(shape=(seq_len,))
    x = Embedding(vocab_size, 128)(note_input)
    x = LSTM(256)(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation="softmax")(x)
    return Model(note_input, output)

classifier = build_mood_classifier(
    vocab_size=vocab_size,
    seq_len=SEQ_LEN,
    num_classes=len(MOODS)
)

In [76]:
classifier.load_weights(CLASSIFIER_WEIGHTS)
print("Mood classifier weights loaded")

Mood classifier weights loaded


In [77]:
def extract_notes_from_midi(midi_path):
    midi = pretty_midi.PrettyMIDI(midi_path)
    notes = []

    for instrument in midi.instruments:
        if instrument.is_drum:
            continue
        for note in instrument.notes:
            notes.append(str(note.pitch))

    return notes


notes = extract_notes_from_midi(GENERATED_MIDI)
print("Total notes extracted:", len(notes))

Total notes extracted: 300


In [78]:
def build_sequences(notes, seq_len):
    encoded = [note_to_idx[n] for n in notes if n in note_to_idx]

    X = []
    for i in range(len(encoded) - seq_len):
        X.append(encoded[i:i + seq_len])

    return np.array(X)


X_gen = build_sequences(notes, SEQ_LEN)
print("X_gen shape:", X_gen.shape)

X_gen shape: (250, 50)


In [79]:
preds = classifier.predict(X_gen, batch_size=256, verbose=0)
avg_probs = np.mean(preds, axis=0)

print("\nAverage mood probabilities:")
for mood, prob in zip(MOODS, avg_probs):
    print(f"{mood}: {prob:.3f}")


Average mood probabilities:
happy: 0.491
sad: 0.319
angry: 0.014
quiet: 0.176


In [80]:
predicted_mood = MOODS[np.argmax(avg_probs)]
confidence = np.max(avg_probs)

print("\nPredicted Mood:", predicted_mood)
print("Confidence:", round(confidence * 100, 2), "%")


Predicted Mood: happy
Confidence: 49.1 %
